In [9]:
# IS_IDS_1.ipynb - FINAL CODE BLOCK (FIXES NOTFITTEDERROR)

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from imblearn.over_sampling import SMOTE
import joblib
import os
import warnings
warnings.filterwarnings("ignore")

# --- 1. DEFINITIONS AND SETUP ---
FLOW_FEATURE_NAMES = [
    'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
    'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
    'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
    'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min',
    'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
    'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
    'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
    'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
    'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
    'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
    'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length',
    'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
    'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count',
    'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count',
    'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size',
    'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Header Length.1',
    'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate',
    'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate',
    'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
    'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
    'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std',
    'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
    'Idle Min'
]

# --- 2. DATA PREPARATION ---
# NOTE: Ensure CICIDS2017_sample.csv is in your C:\Users\HP\Downloads\IS_Project\IS_Project folder!
df = pd.read_csv(r'C:\Users\HP\Downloads\IS_Project\IS_Project\CICIDS2017_sample.csv')

# Sampling Logic
df_minor = df[(df['Label']=='WebAttack')|(df['Label']=='Bot')|(df['Label']=='Infiltration')]
df_BENIGN = df[(df['Label']=='BENIGN')].sample(n=None, frac=0.01, replace=False, random_state=0, axis=0)
df_DoS = df[(df['Label']=='DoS')].sample(n=None, frac=0.05, replace=False, random_state=0, axis=0)
df_PortScan = df[(df['Label']=='PortScan')].sample(n=None, frac=0.05, replace=False, random_state=0, axis=0)
df_BruteForce = df[(df['Label']=='BruteForce')].sample(n=None, frac=0.2, replace=False, random_state=0, axis=0)
df_s = pd.concat([df_BENIGN, df_DoS, df_PortScan, df_BruteForce, df_minor]).sort_index()

# Separate Features and Labels
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(df_s.iloc[:, -1]).astype(int) # CRITICAL: Encoder is fitted here
X = df_s.drop(['Label'], axis=1)

# CRITICAL FIXES FOR INFINITY VALUES
X.replace([np.inf, -np.inf], 0.0, inplace=True)
X.fillna(0.0, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(
    X.values, y, train_size=0.8, test_size=0.2, random_state=0, stratify=y
)

# Scaling 
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(sampling_strategy={4: 1500})
X_train, y_train = smote.fit_resample(X_train, y_train.astype(int))

# --- 3. MODEL TRAINING ---

# Base Models
dt = DecisionTreeClassifier(random_state=0); dt.fit(X_train, y_train)
rf = RandomForestClassifier(random_state=0); rf.fit(X_train, y_train)
et = ExtraTreesClassifier(random_state=0); et.fit(X_train, y_train)
xg = xgb.XGBClassifier(n_estimators=10); xg.fit(X_train, y_train)

# Stacking Meta-Model
dt_train = dt.predict(X_train).reshape(-1, 1); dt_test = dt.predict(X_test).reshape(-1, 1)
rf_train = rf.predict(X_train).reshape(-1, 1); rf_test = rf.predict(X_test).reshape(-1, 1)
et_train = et.predict(X_train).reshape(-1, 1); et_test = et.predict(X_test).reshape(-1, 1)
xg_train = xg.predict(X_train).reshape(-1, 1); xg_test = xg.predict(X_test).reshape(-1, 1)

x_train_stk = np.concatenate((dt_train, et_train, rf_train, xg_train), axis=1)
x_test_stk = np.concatenate((dt_test, et_test, rf_test, xg_test), axis=1)

stk = xgb.XGBClassifier().fit(x_train_stk, y_train)

# --- 4. SAVING ALL FLOW COMPONENTS (8 FILES) ---
SAVE_DIR = '.' 

joblib.dump(scaler, os.path.join(SAVE_DIR, 'ids_minmax_scaler.joblib'))
joblib.dump(stk, os.path.join(SAVE_DIR, 'ids_stacking_meta_model.joblib'))
joblib.dump(dt, os.path.join(SAVE_DIR, 'dt_model.joblib'))
joblib.dump(rf, os.path.join(SAVE_DIR, 'rf_model.joblib'))
joblib.dump(et, os.path.join(SAVE_DIR, 'et_model.joblib'))
joblib.dump(xg, os.path.join(SAVE_DIR, 'xg_model.joblib'))
joblib.dump(labelencoder, os.path.join(SAVE_DIR, 'label_encoder.joblib')) # CORRECT: Saves the fitted object!
joblib.dump(FLOW_FEATURE_NAMES, os.path.join(SAVE_DIR, 'flow_feature_names.joblib')) 

print("\n--- PHASE 1 COMPLETE: Flow Model Components Saved. ---")


--- PHASE 1 COMPLETE: Flow Model Components Saved. ---
